# MATLAB Control Panel for Databricks

Use this notebook in a Databricks environment to List, Start, Stop MATLAB sessions on a compute node.


In [ ]:
import sys

sys.path.append("mwhelpers")
from mwhelpers import mwi
# from mwhelpers import mwi_test as mwi

import ipywidgets as widgets
from IPython.display import display

num_row = 2
num_col = 2
grid = widgets.GridspecLayout(
    num_row, num_col, grid_gap="1px", justify_content="center", align_items="center"
)

## Section Header : Title
title_html = widgets.HTML(
    """
    <div style="display: flex; align-items: center; justify-content: center; padding: 1px;">
        <h1 style="margin: 0;">MATLAB Control Panel</h1>
    </div>
    """
)
current_row = 0
grid[current_row, 0:] = widgets.HBox(
    [title_html],
    layout=widgets.Layout(justify_content="center", align_items="center"),
)
## Section

## Username  | Cluster Name
username_hbox = widgets.Text(
    value=mwi.get_user_name(),
    style={"description_width": "initial"},
    required=True,
    description_allow_html=True,
    description="<b>Username:<b>",
)
current_row += 1
grid[current_row, 0] = username_hbox

clustername_html = widgets.HTML(
    description="Cluster Name:",
    value="<b> Unavailable</b>",
    style={"description_width": "initial"},
)
grid[current_row, 1] = clustername_html

## Display the APP
display(grid)
context = mwi.get_databricks_context()
cluster_name = mwi.get_cluster_name()
if not cluster_name:
    clustername_html.value = "<b> Serverless/Invalid </b>"
    display(
        widgets.HTML(
            "<div style='color:red; display: flex; align-items: center; justify-content: center; border: 1px solid black; padding: 1px;'><b>Connect to a supported cluster to continue.</b></div>",
        )
    )
else:
    # Proceed with building the rest of the app only if we are in a valid cluster.
    clustername_html.value = f"<b> {cluster_name} </b>"

    ## Create data for Configuration Tab
    matlabroot_text = widgets.Text(
        description="MATLAB Root:",
        value=mwi.get_matlab_root(),
        style={"description_width": "initial"},
        disabled=True,
    )
    matlab_version_text = widgets.Text(
        description="MATLAB Version:",
        value=mwi.get_matlab_version(),
        style={"description_width": "initial"},
        disabled=True,
    )
    ## PSP Check box
    psp_checkbox = widgets.Checkbox(
        description="Use MATLAB Interface for Databricks",
        button_style="info",
        value=True,
        layout=widgets.Layout(height="50px", width="auto", border=""),
    )

    installed_toolboxes_selectmultiple = widgets.SelectMultiple(
        options=mwi.get_installed_toolboxes(),
        rows=10,
        disabled=True,
    )
    installed_toolboxes_vbox = widgets.VBox(
        [
            widgets.Label("These toolboxes are installed on this cluster."),
            installed_toolboxes_selectmultiple,
        ]
    )

    installed_toolboxes_accordion = widgets.Accordion(
        children=[installed_toolboxes_vbox]
    )
    installed_toolboxes_accordion.set_title(0, "Installed Toolboxes")

    available_toolboxes_selectmultiple = widgets.SelectMultiple(
        options=mwi.get_toolboxes_available_for_install(),
        rows=10,
        disabled=False,
    )
    available_toolboxes_vbox = widgets.VBox(
        [
            widgets.Label("Selection will be installed in the next new session."),
            widgets.Label("NOTE: Installation permanently modifies the cluster"),
            available_toolboxes_selectmultiple,
        ]
    )

    available_toolboxes_accordion = widgets.Accordion(
        children=[available_toolboxes_vbox]
    )
    available_toolboxes_accordion.set_title(0, "Select Toolboxes to Install")

    configuration_app = widgets.AppLayout(
        header=widgets.VBox(
            [widgets.HBox([matlabroot_text, matlab_version_text]), psp_checkbox]
        ),
        left_sidebar=installed_toolboxes_accordion,
        center=None,
        right_sidebar=available_toolboxes_accordion,
        footer=None,
    )

    ## Create data for Access Tab
    app_outputs = widgets.Output(layout={"border": "1px solid black"})
    app_outputs.append_stdout(" ...")
    logs_textarea_label = widgets.Label("Outputs:")

    available_matlab_sessions_selectmultiple = widgets.SelectMultiple(
        options=mwi.get_running_matlab_proxy_servers(),
        rows=10,
        disabled=False,
        layout=widgets.Layout(width="150px"),
    )
    available_matlab_sessions_vbox = widgets.VBox(
        [
            widgets.Label("MATLAB Sessions:"),
            available_matlab_sessions_selectmultiple,
        ],
        layout=widgets.Layout(min_width="175px"),
    )

    connect_matlab_button = widgets.Button(
        description="Connect to MATLAB",
        button_style="success",
    )

    stop_matlab_button = widgets.Button(
        description="Stop MATLAB",
        button_style="danger",
    )
    start_matlab_button = widgets.Button(
        description="Start New MATLAB",
        button_style="Primary",
    )
    refresh_matlab_button = widgets.Button(
        description="Refresh List",
        button_style="info",
    )

    clear_logs_button = widgets.Button(
        description="Clear Logs",
        button_style="",
    )

    list_matlab_app = widgets.AppLayout(
        header=widgets.Label(
            "Use the controls below to start/connect/stop MATLAB sessions."
        ),
        left_sidebar=available_matlab_sessions_vbox,
        center=widgets.VBox(
            [
                widgets.Label("Controls:"),
                start_matlab_button,
                connect_matlab_button,
                stop_matlab_button,
                refresh_matlab_button,
                clear_logs_button,
            ],
            layout=widgets.Layout(align_items="center", min_width="200px"),
        ),
        right_sidebar=widgets.VBox(
            [logs_textarea_label, app_outputs],
            layout=widgets.Layout(min_width="400px", max_height="400px"),
        ),
        pane_widths=[1, 2, 5],
        pane_heights=[1, 5, 5],
    )

    @app_outputs.capture(clear_output=True)
    def list_matlab_sessions(b):
        display(list_matlab_app)

    @app_outputs.capture(clear_output=True)
    def print100hello(b):
        for i in range(100):
            print(f"{i}. hello")

    tabs = widgets.Tab(children=[list_matlab_app, configuration_app])
    tabs.set_title(0, "MATLAB Sessions")
    tabs.set_title(1, "Configuration")

    # display(tabs)

    ## App Functionality begins here:

    def clear_logs(b):
        app_outputs.clear_output(wait=True)
        with app_outputs:
            print("...")

    clear_logs_button.on_click(clear_logs)

    def refresh_matlab_sessions(b):
        available_matlab_sessions_selectmultiple.options = (
            mwi.get_running_matlab_proxy_servers()
        )

    @app_outputs.capture(clear_output=True)
    def stop_matlab_session(b):
        selected_sessions = available_matlab_sessions_selectmultiple.value
        if selected_sessions:
            for session in selected_sessions:
                mwi.stop_matlab_session(session)
                print(f"Stopped MATLAB session: {session}")
        else:
            print("No sessions selected to stop.")
        available_matlab_sessions_selectmultiple.options = (
            mwi.get_running_matlab_proxy_servers()
        )

    def connect_to_matlab_sessionid(session_id):
        URL = mwi.get_url_to_matlab(session_id, context)
        window_open_command = f"window.open('{URL}', '_blank');"
        button_text = widgets.Label(f"Generating link to MATLAB session: {session_id}")
        button_html = f"""<button style="background-color: green; color: white;" onclick="{window_open_command}">Open MATLAB Session {session_id}</button>"""
        display(
            widgets.VBox(
                [
                    button_text,
                    widgets.HTML(button_html),
                ]
            )
        )

    @app_outputs.capture(clear_output=True)
    def connect_to_matlab_session(b):
        display_header = widgets.Label("------------------")
        display(display_header)
        selected_sessions = available_matlab_sessions_selectmultiple.value
        if selected_sessions:
            for session_id in selected_sessions:
                connect_to_matlab_sessionid(session_id)
        else:
            print("No sessions selected to connect.")
        display(display_header)

    @app_outputs.capture(clear_output=True)
    def start_matlab_session(b):
        start_matlab_button.disabled = True
        app_outputs.clear_output(wait=True)

        toolboxes_to_install = available_toolboxes_selectmultiple.value
        configure_psp = psp_checkbox.value
        print("Starting MATLAB session...")
        session_id = mwi.start_matlab_session(configure_psp, toolboxes_to_install)
        available_matlab_sessions_selectmultiple.options = (
            mwi.get_running_matlab_proxy_servers()
        )
        print("Started MATLAB session.")
        connect_to_matlab_sessionid(session_id)
        start_matlab_button.disabled = False

    start_matlab_button.on_click(start_matlab_session)
    connect_matlab_button.on_click(connect_to_matlab_session)
    stop_matlab_button.on_click(stop_matlab_session)
    refresh_matlab_button.on_click(refresh_matlab_sessions)

    refresh_matlab_sessions(None)
    display(tabs)

GridspecLayout(children=(HBox(children=(HTML(value='\n    <div style="display: flex; align-items: center; just…